# Custom VGG16 Model for Tiny ImageNet Classification
 **Brief Overview-**
* The dataset contains square images of 64x64 pixels.
* Each image belongs to exactly one out of 200 categories.
* The training set contains 90,000 images, 450 from each category.
* The validation and test sets have 10,000 images each, 50 from each category.
* Modified VGG16 model is used for training.
* Image augmentation techniques are used for avoiding overfitting and better accuracy.

# Installing and Importing Libraries
* Tensorflow and Keras frameworks are used for this project.
* Keras-Contrib library is for CyclicLR callback.
* Keract library is for visualizing the layer outputs.

In [ ]:
!pip install git+https://github.com/keras-team/keras-contrib.git
!pip install keract

In [ ]:
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import keract
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_contrib.callbacks import CyclicLR

## Dataset

In [ ]:
train_data_dir = '../input/imagedetect/train'  
validation_data_dir = '../input/imagedetect/val' 
test_data_dir = '../input/imagedetect/test'

In [ ]:
img_width, img_height = 64, 64 
channels = 3
batch_size = 64

In [ ]:
val_data = pd.read_csv(validation_data_dir + '/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
)

datagen = ImageDataGenerator(rescale= 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(  
    train_data_dir,  
    target_size= (img_width, img_height), 
    color_mode= 'rgb',
    batch_size= batch_size,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 42
) 

valid_generator = datagen.flow_from_dataframe(
    dataframe= val_data, 
    directory= validation_data_dir + '/images',
    x_col= 'File', 
    y_col= 'Class', 
    target_size= (img_width, img_height),
    color_mode= 'rgb', 
    class_mode= 'categorical', 
    batch_size= batch_size, 
    shuffle= True, 
    seed= 42
)

test_generator = datagen.flow_from_directory(  
    test_data_dir,  
    target_size= (img_width, img_height), 
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= None,
    shuffle= False, 
)

In [ ]:
num_classes = len(train_generator.class_indices)  
train_labels = train_generator.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = valid_generator.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_generator.filenames)  
nb_valid_samples = len(valid_generator.filenames)
nb_test_samples = len(test_generator.filenames)

In [ ]:
img = load_img('../input/imagedetect/train/n02279972/images/n02279972_28.JPEG')
data = img_to_array(img)
samples = expand_dims(data, 0)
it = train_datagen.flow(samples, batch_size=1)

for i in range(9):
    plt.subplot(330 + 1 + i)
    batch = it.next()
    image = batch[0]
    plt.imshow(image)

plt.savefig('augmented_image.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

# Model
* Modified VGG16 CNN architecture is used for the problem.
* Pretrained on the 'ImageNet' dataset.

In [ ]:
vgg16 = applications.VGG16(include_top= False, input_shape= (img_width, img_height, channels), weights= 'imagenet')
vgg16.summary()

In [ ]:
model = Sequential()

for layer in vgg16.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable= False

model.add(Flatten(input_shape= (2, 2, 512)))

model.add(Dense(512, activation= 'relu', name= 'FC1'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(512, activation= 'relu', name= 'FC2'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(200, activation= 'softmax', name= 'FC3'))

model.summary()

# Baseline Model Training
* Cyclical LR in range of 1e-4 to 6e-4 with a step size of 1404.

In [ ]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.0001, epsilon= 1e-08), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [ ]:
my_callbacks = [
    EarlyStopping(monitor= 'val_accuracy', mode= 'auto', patience=2),
    CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=1404., mode= 'triangular2'),
    ModelCheckpoint(filepath= 'baseline_model.h5', monitor= 'val_accuracy', save_best_only= True, mode= 'auto')
]

In [ ]:
history = model.fit(
    train_generator, 
    epochs= 30,
    steps_per_epoch = nb_train_samples//batch_size, 
    validation_data = valid_generator, 
    validation_steps = nb_valid_samples//batch_size,
    verbose = 2, 
    callbacks = my_callbacks,
    shuffle = True
)

In [ ]:
(eval_loss, eval_accuracy) = model.evaluate(valid_generator, batch_size= batch_size, verbose= 1)
print('Validation Loss: ', eval_loss)
print('Validation Accuracy: ', eval_accuracy)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy','Validation Accuracy'])
plt.savefig('base_acc_epoch.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss','Validation Loss'])
plt.savefig('base_loss_epoch.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

plt.plot(history.history['lr'])
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')
plt.legend(['Learning Rate'])
plt.savefig('base_lr_epoch.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

# Finetuning
* Cyclical LR in range of 1e-5 to 6e-5 with a setp size of 1200.
* Further, reducing cyclical LR in the range of 6e-6 to 6e-6, keeping the step size same.

In [ ]:
model.trainable= True
model.compile(optimizer= keras.optimizers.Adam(1e-5), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [ ]:
my_callbacks = [
    CyclicLR(base_lr= 0.00001, max_lr= 0.00006, step_size= 1200., mode= 'triangular2'),
    ModelCheckpoint(filepath= 'finetuned_model_v1.h5', monitor= 'val_accuracy', save_best_only= True, mode= 'auto')
]

In [ ]:
history_1= model.fit(
    train_generator, 
    epochs= 15, 
    steps_per_epoch= nb_train_samples//batch_size, 
    validation_data= valid_generator, 
    validation_steps= nb_valid_samples//batch_size,
    verbose= 2, 
    callbacks= my_callbacks
)

In [ ]:
(eval_loss, eval_accuracy) = model.evaluate(valid_generator, batch_size= batch_size, verbose= 1)
print('Validation Loss: ', eval_loss)
print('Validation Accuracy: ', eval_accuracy)

In [ ]:
plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy','Validation Accuracy'])
plt.savefig('finetune_acc_epoch_v1.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss','Validation Loss'])
plt.savefig('finetuned_loss_epoch_v1.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

plt.plot(history_1.history['lr'])
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')
plt.legend(['Learning Rate'])
plt.savefig('finetuned_lr_epoch_v1.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

In [ ]:
model.compile(optimizer= keras.optimizers.Adam(1e-6), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [ ]:
my_callbacks = [
    CyclicLR(base_lr=0.000001, max_lr=0.000006, step_size=1200., mode= 'triangular2'),
    ModelCheckpoint(filepath= 'finetuned_model_v2.h5', monitor= 'val_accuracy', save_best_only= True, mode= 'auto')
]

In [ ]:
history_2 = model.fit(
    train_generator, 
    epochs= 15, 
    steps_per_epoch= nb_train_samples//batch_size, 
    validation_data= valid_generator, 
    validation_steps= nb_valid_samples//batch_size,
    verbose= 2, 
    callbacks= my_callbacks
)

In [ ]:
(eval_loss, eval_accuracy) = model.evaluate(valid_generator, batch_size= batch_size, verbose= 1)
print('Validation Loss: ', eval_loss)
print('Validation Accuracy: ', eval_accuracy)

In [ ]:
plt.plot(history_2.history['accuracy'])
plt.plot(history_2.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy','Validation Accuracy'])
plt.savefig('finetune_acc_epoch_v2.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss','Validation Loss'])
plt.savefig('finetuned_loss_epoch_v2.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

plt.plot(history_2.history['lr'])
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')
plt.legend(['Learning Rate'])
plt.savefig('finetuned_lr_epoch_v2.png', transparent= True, bbox_inches= 'tight', dpi= 900)
plt.show()

# Predictions on Test Set

In [ ]:
def getListKeys(dict): 
    list = [] 
    for key in dict.keys(): 
        list.append(key) 
    return np.asarray(list)

labels = getListKeys(train_generator.class_indices)

filenames = np.array([])
file_names = test_generator.filenames
for file in file_names:
    temp = file.split('/')
    filenames = np.append(filenames, temp[1])

In [ ]:
predict = model.predict(test_generator)
predicted_class_indices= np.argmax(predict,axis=1)
predictions= np.asarray([labels[i] for i in predicted_class_indices])

test_df = pd.DataFrame()
test_df['file_name'] = filenames
test_df['category'] = predictions
test_df.to_csv('test_predictions.csv', index= False)

# Visualizing Layers

In [ ]:
image = load_img('../input/imagedetect/train/n02124075/images/n02124075_360.JPEG', target_size= (64, 64))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
y_hat = model.predict(image)

In [ ]:
activations= keract.get_activations(model, image, layer_names= None, nodes_to_evaluate= None, output_format= 'simple', auto_compile= True)
keract.display_activations(activations, save= False)